#### EDA - Explorative data analysis

Im folgenden werden Korrekturen an den Daten vorgenommen.


In [105]:
# Bibliotheken importieren
import pandas as pd
import altair as alt
from pathlib import Path

In [106]:
# CSV Einlesen 
parent_path = str(Path().resolve().parent.parent) + "/"
data_path = "data/"
processed_path = "processed/"
processed_file = "processed_data.csv"

df = pd.read_csv(parent_path + data_path + processed_path + processed_file, encoding= 'unicode_escape')

#df.info()
#df.describe()

In [107]:
# Weitere Korrekturen

# Doppelte Zeilen entfernen
df = df.drop_duplicates()

# Fehlende Werte auslesen
df.isna().sum()

# Fehlende Werte löschen
df = df.dropna(subset=['CustomerID'])
df = df.dropna(subset=['Description'])

In [108]:
# Prüfung auf ungewöhnliche Werte
print(df.describe())

            Quantity      UnitPrice     CustomerID     TotalPrice
count  399823.000000  399823.000000  399823.000000  399823.000000
mean       12.225617       2.923285   15288.460319      20.689437
std       250.794906       4.537682    1710.619396     425.444611
min    -80995.000000       0.000000   12346.000000 -168469.600000
25%         2.000000       1.250000   13959.000000       4.250000
50%         5.000000       1.950000   15150.000000      11.580000
75%        12.000000       3.750000   16791.000000      19.500000
max     80995.000000     649.500000   18287.000000  168469.600000


In [109]:
# Sortieren des DataFrame nach Gesamtpreis in absteigender Reihenfolge
df_sorted = df.sort_values(by='TotalPrice', ascending=False)

print(df_sorted.head(10))

       InvoiceNo StockCode                          Description  Quantity  \
537633    581483     23843          PAPER CRAFT , LITTLE BIRDIE     80995   
61349     541431     23166       MEDIUM CERAMIC TOP STORAGE JAR     74215   
221402    556444     22502       PICNIC BASKET WICKER 60 PIECES        60   
346350    567423     23243  SET OF TEA COFFEE SUGAR TINS PANTRY      1412   
52480     540815     21108   FAIRY CAKE FLANNEL ASSORTED COLOUR      3114   
159660    550461     21108   FAIRY CAKE FLANNEL ASSORTED COLOUR      3114   
419258    573003     23084                   RABBIT NIGHT LIGHT      2400   
52478     540815    85123A   WHITE HANGING HEART T-LIGHT HOLDER      1930   
159656    550461    85123A   WHITE HANGING HEART T-LIGHT HOLDER      1930   
52541     540818     48185                   DOORMAT FAIRY CAKE       670   

             InvoiceDate  UnitPrice  CustomerID         Country  TotalPrice  
537633    12/9/2011 9:15       2.08     16446.0  United Kingdom   168469.6

In [110]:
# Warenkorb pro Bestellung
df_invoice = df.groupby(['InvoiceNo'])['TotalPrice'].sum().reset_index()

# Zugunsten der Visualisierungen negative Werte entfernen
df_invoice.drop(df_invoice[df_invoice['TotalPrice'] < 0].index, inplace=True)

#df_invoice.head()


In [112]:
# Vermutlich Outlier

df_sorted = df_sorted.head(10)

df_sorted

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
537633,581483,23843,"PAPER CRAFT , LITTLE BIRDIE",80995,12/9/2011 9:15,2.08,16446.0,United Kingdom,168469.60
61349,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,1/18/2011 10:01,1.04,12346.0,United Kingdom,77183.60
221402,556444,22502,PICNIC BASKET WICKER 60 PIECES,60,6/10/2011 15:28,649.50,15098.0,United Kingdom,38970.00
346350,567423,23243,SET OF TEA COFFEE SUGAR TINS PANTRY,1412,9/20/2011 11:05,5.06,17450.0,United Kingdom,7144.72
52480,540815,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,3114,1/11/2011 12:55,2.10,15749.0,United Kingdom,6539.40
159660,550461,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,3114,4/18/2011 13:20,2.10,15749.0,United Kingdom,6539.40
419258,573003,23084,RABBIT NIGHT LIGHT,2400,10/27/2011 12:11,2.08,14646.0,Netherlands,4992.00
52478,540815,85123A,WHITE HANGING HEART T-LIGHT HOLDER,1930,1/11/2011 12:55,2.55,15749.0,United Kingdom,4921.50
159656,550461,85123A,WHITE HANGING HEART T-LIGHT HOLDER,1930,4/18/2011 13:20,2.40,15749.0,United Kingdom,4632.00
52541,540818,48185,DOORMAT FAIRY CAKE,670,1/11/2011 12:57,6.75,15749.0,United Kingdom,4522.50


In [113]:
# Identifizieren von Retouren
df_C = df[df['InvoiceNo'].str.contains('C')]

df_C

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
152,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,12/1/2010 9:49,4.65,15311.0,United Kingdom,-4.65
233,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,12/1/2010 10:24,1.65,17548.0,United Kingdom,-19.80
234,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom,-6.96
235,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom,-6.96
236,C536391,21980,PACK OF 12 RED RETROSPOT TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom,-6.96
...,...,...,...,...,...,...,...,...,...
537660,C581490,22178,VICTORIAN GLASS HANGING T-LIGHT,-12,12/9/2011 9:57,1.95,14397.0,United Kingdom,-23.40
537661,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,12/9/2011 9:57,0.83,14397.0,United Kingdom,-9.13
538922,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,12/9/2011 11:57,10.95,15311.0,United Kingdom,-54.75
538923,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,12/9/2011 11:58,1.25,17315.0,United Kingdom,-1.25


In [114]:
# Outlier mit returns matchen

df_new = pd.DataFrame(columns=df_C.columns)  # Datenframe mit den gleichen Spalten wie df_C erstellen

for i, row in df_sorted.iterrows():
    # Überprüfen, ob die Werte in der Zeile den entsprechenden Werten in df_C entsprechen
    mask = (df_C['Description'] == row['Description']) & (df_C['StockCode'] == row['StockCode']) & (df_C['UnitPrice'] == row['UnitPrice']) & (df_C['CustomerID'] == row['CustomerID']) & (df_C['Country'] == row['Country'])
    # Wenn es in df_C eine Zeile gibt, die übereinstimmt, wird die Zeile von df_sorted zu df_new hinzugefügt
    if mask.any():
        df_new = df_new.append(row)


for index, row in df_sorted.iterrows():
    # Boolesche Maske erstellen um Zeilen in df_C auszuwählen, die der aktuellen Zeile in df_sorted entsprechen
    mask = (df_C['Description'] == row['Description']) & (df_C['StockCode'] == row['StockCode']) & (df_C['UnitPrice'] == row['UnitPrice']) & (df_C['CustomerID'] == row['CustomerID']) & (df_C['Country'] == row['Country'])
    
    # Passende Zeilen aus df_C auswählen
    matching_rows = df_C[mask]
    
    # Passende Zeilen zu df_new hinzufügen
    df_new = df_new.append(matching_rows, ignore_index=True)


df_new

C:\Users\mcwil\AppData\Local\Temp\ipykernel_6420\400063773.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_new = df_new.append(row)
C:\Users\mcwil\AppData\Local\Temp\ipykernel_6420\400063773.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_new = df_new.append(row)
C:\Users\mcwil\AppData\Local\Temp\ipykernel_6420\400063773.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_new = df_new.append(row)
C:\Users\mcwil\AppData\Local\Temp\ipykernel_6420\400063773.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_new = df_new.append(row)
C:\Users\mcwil\AppData\Local\Temp\ipykernel_6420\400063773.py:10: FutureWarning: The

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,581483,23843,"PAPER CRAFT , LITTLE BIRDIE",80995,12/9/2011 9:15,2.08,16446.0,United Kingdom,168469.6
1,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,1/18/2011 10:01,1.04,12346.0,United Kingdom,77183.6
2,540815,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,3114,1/11/2011 12:55,2.10,15749.0,United Kingdom,6539.4
3,550461,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,3114,4/18/2011 13:20,2.10,15749.0,United Kingdom,6539.4
4,540815,85123A,WHITE HANGING HEART T-LIGHT HOLDER,1930,1/11/2011 12:55,2.55,15749.0,United Kingdom,4921.5
5,540818,48185,DOORMAT FAIRY CAKE,670,1/11/2011 12:57,6.75,15749.0,United Kingdom,4522.5
6,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,12/9/2011 9:27,2.08,16446.0,United Kingdom,-168469.6
7,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,1/18/2011 10:17,1.04,12346.0,United Kingdom,-77183.6
8,C550456,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,-3114,4/18/2011 13:08,2.10,15749.0,United Kingdom,-6539.4
9,C550456,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,-3114,4/18/2011 13:08,2.10,15749.0,United Kingdom,-6539.4


In [115]:
# Retouren aus ursprünglichem DataFrame entfernen
df = df[~df['InvoiceNo'].isin(df_new['InvoiceNo'])]

df.describe()

,Quantity,UnitPrice,CustomerID,TotalPrice
count,399804.000000,399804.000000,399804.000000,399804.000000
mean,12.203652,2.923293,15288.451969,20.636554
std,48.834679,4.537774,1710.643099,91.753865
min,-9360.000000,0.000000,12347.000000,-3825.360000
25%,2.000000,1.250000,13959.000000,4.250000
50%,5.000000,1.950000,15150.000000,11.580000
75%,12.000000,3.750000,16791.000000,19.500000
max,12540.000000,649.500000,18287.000000,38970.000000


In [116]:
# Korrekturen
# Nur positive Menge
#df = df[df['Quantity'] > 0]

# Nur positiver Preis
#df = df[df['UnitPrice'] > 0]

#df.describe()

In [121]:
# Häufigkeit der Werte in StockCode
df_stockcode_counts = df['StockCode'].value_counts().reset_index()
df_stockcode_counts.columns = ['StockCode', 'Frequency']
df_stockcode_counts_sorted = df_stockcode_counts.sort_values(by='Frequency', ascending=False)

df_stockcode_counts_sorted_short = df_stockcode_counts_sorted.head(100)

# Barplot erstellen
chart = alt.Chart(df_stockcode_counts_sorted_short).mark_bar().encode(
x='StockCode',
y='Frequency'
).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=22,
    font="Arial",
    color="black",
    anchor="start"
).configure_axis(
    labelFont="Arial",
    titleFont="Arial",
    labelFontSize=14,
    titleFontSize=16,
    titleFontWeight="normal",
    labelColor="grey",
    titleColor="grey"
).properties(
    title={"text":["StockCode distribution"], "subtitle":["How often each product occurs"]},
)

chart


c:\Users\mcwil\anaconda3\envs\myenv\lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)